In [1]:
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
from sklearn import metrics as sk_metrics
import numpy as np

In [2]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.CenterCrop((200,1500)),
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir, data_transforms['test'])
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes

In [3]:
data_prep = DataPreparation('../data/figures/test')
data, size = data_prep.create_dataloaders(16, False, 4)

In [4]:
model_path = Path().cwd().parents[0] / "models/resnet50_d_28_t_20_49.pth"
model = torch.load(model_path)
model = model.eval()

In [5]:
y_pred = []
y_true = []

for inputs, labels in data['test']:
    inputs = inputs.to(0)
    labels = labels.to(0)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    y_pred.append(preds.cpu().numpy())
    y_true.append(labels.cpu().numpy())


In [6]:
y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

In [7]:
acc = sk_metrics.accuracy_score(y_pred, y_true)
f1 = sk_metrics.f1_score(y_pred, y_true)

In [8]:
acc, f1 

(0.8923924399278184, 0.9363947678661653)

In [16]:
from torchvision import models

In [24]:
model = models.inception_v3(pretrained=True).Mixed_7a.parameters()

In [23]:
{"params": model.Mixed_7a.parameters(), "lr": 1e-6},
{"params": model.Mixed_7b.parameters(), "lr": 1e-6},
{"params": model.Mixed_7c.parameters(), "lr": 1e-6}
{"params": model.fc.parameters(), "lr": 1e-4}

<bound method Module.children of Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(

In [25]:
import shutil

In [26]:
raw_fig = Path.cwd().parents[0] / "data/raw_figures"
labels = Path.cwd().parents[0] / "data/labels"

In [30]:
for filename in labels.iterdir():
    shutil.copy(filename, raw_fig / filename.stem)